In [ ]:
def clean_health_data(input_file_path, output_file_path, drop_cols_thresh=0.4):

    import pandas as pd
    import numpy as np
    from sklearn.impute import SimpleImputer
    from sklearn.preprocessing import LabelEncoder, OneHotEncoder

#Def clean_health_data(input_file_path, output_file_path, drop_cols_thresh=0.4):
#Load input data
    data = pd.read_excel(input_file_path, sheet_name=1, engine='openpyxl')

#Convert 'Y' and 'N' values to 1 and 0 respectively
    data = data.replace({'Y': 1, 'N': 0})

#Group rare values in 'Ntm_Speciality' column as 'OTHER' if the percentage of that category is less than 1%
    ntm_spec_counts = data['Ntm_Speciality'].value_counts(normalize=True)
    rare_ntm_spec = ntm_spec_counts[ntm_spec_counts < 0.01].index
    data['Ntm_Speciality'] = data['Ntm_Speciality'].replace(rare_ntm_spec, 'OTHER')

#Remove columns with missing value percentage above a threshold
    data = data.dropna(thresh=int(data.shape[0] * (1 - drop_cols_thresh)), axis=1)

#Replace 'Unknown' values in 'Tscore_Bucket_During_Rx' column with corresponding values from 'Tscore_Bucket_Prior_Ntm' column
    data['Tscore_Bucket_During_Rx'] = np.where(data['Tscore_Bucket_During_Rx'] == 'Unknown', data['Tscore_Bucket_Prior_Ntm'], data['Tscore_Bucket_During_Rx'])

#Convert '> -2.5' and '<= -2.5' values to 1 and 0 respectively
    data = data.replace({'>-2.5': 1, '<=-2.5': 0})

#Convert 'VLR_LR' and 'HR_VHR' values to 1 and 0 respectively
    data = data.replace({'VLR_LR': 1, 'HR_VHR': 0})

#Replace 'Unknown' and 'Other/Unknown' values with NaN
    data = data.replace(["Other/Unknown", "Unknown"], np.nan)

#Replace all missing values in each column with the mode of that column
    imputer = SimpleImputer(strategy='most_frequent')
    data = pd.DataFrame(imputer.fit_transform(data), columns=data.columns)

#Convert the 'Age_Bucket' column to numeric values based on the provided mapping
    age_bucket_mapping = {'>75': 0, '65-75': 1, '55-65': 2, '<55': 3}
    data['Age_Bucket'] = data['Age_Bucket'].map(age_bucket_mapping)

#Detect and remove outliers using IQR
    Q1 = data.quantile(0.25)
    Q3 = data.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = data[((data < lower_bound) | (data > upper_bound)).any(axis=1)]
    data = data[~((data < lower_bound) | (data > upper_bound)).any(axis=1)]

    # Export cleaned data to CSV file
    data.to_csv(output_file_path, index=False)
    
clean_health_data(r'C:\Users\shrut\Desktop\Data Glacier Stuff\Healthcare_dataset.xlsx', r'Clean_file.csv')

In [2]:
def altCleaning(input_file_path):
    """
    Perform data cleaning and pre-processing on an input Excel file.
    
    Args:
    - input_file_path: str, path to the input Excel file
    
    Returns:
    - tuple of numpy arrays:
      - X: 2D array of shape (n_samples, n_features), pre-processed features
      - Y: 1D array of shape (n_samples,), encoded target labels
    """
    import pandas as pd
    import numpy as np
    from sklearn.impute import SimpleImputer
    from sklearn.preprocessing import LabelEncoder, OneHotEncoder

#load the input data
    data = pd.read_excel(input_file_path, sheet_name=1, engine='openpyxl')

#Remove unnecessary columns
    data = data.drop(columns=['Patient_ID', 'Persistency_Flag'])

#Fill missing values with the most frequent value for each column
    imputer = SimpleImputer(strategy="most_frequent")
    features = imputer.fit_transform(data)

#Encode the target variable as numeric labels
    label_encoder = LabelEncoder()
    target = label_encoder.fit_transform(data.Persistency_Flag)

#Encode the categorical features as binary vectors
    ohe = OneHotEncoder()
    features = ohe.fit_transform(features[:, :-1]).toarray()

#Return the pre-processed features and the encoded labels
    return features, target